In [0]:
!wget http://www.openslr.org/resources/12/dev-clean.tar.gz
!tar xzf dev-clean.tar.gz
!sudo apt-get install tree -y
!pip install soundfile
!pip install pydub

In [0]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import soundfile as sf
import os
from pydub import AudioSegment
import numpy as np
import random
from itertools import permutations
import numpy as np
import random
import sys
import io
import os
import glob
import IPython
%matplotlib inline
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Flatten
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape, Concatenate,concatenate, Average
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow as tf

In [0]:
def graph_spectrogram(wav_file):
    rate, data = get_wav_info(wav_file)
    nfft = 200 # Length of each window segment
    fs = 8000 # Sampling frequencies
    noverlap = 120 # Overlap between windows
    nchannels = data.ndim
    if nchannels == 1:
        pxx, freqs, bins, im = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, freqs, bins, im = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

# Load a wav file
def get_wav_info(wav_file):
    rate, data = wavfile.read(wav_file)
    return rate, data

def process_sliding_window(flac_file, processed_directory_path,filename):
    window_size = 1600
    step_size = 800
    actual_idx = 0
    amount_of_files = 0
    new_base_directory = processed_directory_path +filename[:-5]+"/"
    new_base_path = (new_base_directory+filename)[:-5]

    if not os.path.exists(new_base_directory):
        	os.makedirs(new_base_directory)

    while(len(flac_file) > actual_idx + window_size):
        flac_file[actual_idx:actual_idx+window_size].export(new_base_path+'_'+str(amount_of_files) + '.wav',format='wav')
        amount_of_files += 1
        actual_idx += step_size
    if (actual_idx + window_size - len(flac_file) > window_size/2 and len(flac_file) > window_size):
        flac_file[-window_size:].export(new_base_path+'_'+str(amount_of_files) + '.wav',format='wav')
        

def make_processed_dataset(libri_speech_path,processed_libri_speech_path):
    dev_path = libri_speech_path + "dev-clean/"
    processed_dev_path = processed_libri_speech_path + "dev-clean/"
    voice_count = 0
    for voice in (os.listdir(dev_path)):
        voice_path = dev_path + voice + "/"
        for chapter in (os.listdir(voice_path)):
            chapter_path = voice_path + chapter + "/"
            for filename in os.listdir(chapter_path):
                processed_directory_path = processed_dev_path + voice + "/" + chapter + "/" 
                if filename.endswith("flac"):
                    flac_file = AudioSegment.from_file(chapter_path + filename, "flac")
                    process_sliding_window(flac_file,processed_directory_path,filename)
        voice_count += 1
        print ("actual progress", str (int(voice_count/len(os.listdir(dev_path))*100)), ('%'))
        
def process_16_dataset_main():
    dataset_base_path = "./"
    libri_speech_path = dataset_base_path + "LibriSpeech/"
    processed_libri_speech_path = dataset_base_path + "16_LibriSpeech/"
    make_processed_dataset(libri_speech_path, processed_libri_speech_path)


def make_numpy_XY(libri_speech_path):
    dev_path = libri_speech_path + "dev-clean/"
    voice_count = 0

    X = []
    Y = []
    for voice in (os.listdir(dev_path)):
        voice_path = dev_path + voice + "/"
        count = 0     
        X_5 = []
        for chapter in (os.listdir(voice_path)):
            chapter_path = voice_path + chapter + "/"
            for uterrance in os.listdir(chapter_path):
              uterrance_path = chapter_path + uterrance + "/"
              for filename in os.listdir(uterrance_path):
                if filename.endswith("wav"):
                    count += 1
                    wav_file = graph_spectrogram(uterrance_path + filename)
                    X_5.append(wav_file)
                if (count % 5 == 0 and count != 0):
                    X.append(X_5)
                    X_5 = []
                    Y.append(voice)
                if (count >= 100):
                    break
            if count>=100:
                break
        voice_count+=1
        print ("actual progress", str (int(voice_count/len(os.listdir(dev_path))*100)), ('%'))
    return X,Y

def process_numpy_XY():
    dataset_base_path = "./"
    processed_libri_speech_path = dataset_base_path + "16_LibriSpeech/"
    X,Y = make_numpy_XY(processed_libri_speech_path)
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

def reshape_X(X):
    examples = X.shape[0]
    dimension = X.shape[1]
    time = X.shape[2]
    freq = X.shape[3]
    new_X = np.zeros((dimension,examples,time,freq))
    for m in range(examples):
        for d in range(dimension):
            new_X[d][m] = X[m][d]
    return new_X
    
    
def load_numpy_XY():
    X_path = './preprocessed/X_total.npy'
    Y_path = './preprocessed/Y_total.npy'
    X = np.load(X_path)
    Y = np.load(Y_path)
    return X,Y

In [0]:
print ('processing dataset:')
process_16_dataset_main()
print ('----------------')
print ('Making XY:')
X,Y = process_numpy_XY()

In [0]:
def five_average_model(input_shape,base_model):
    model_inputs = []
    model_results = []
    for i in range(5):
        X_input = Input(shape = input_shape, name = "Input"+ str(i))
        model_inputs.append(X_input)
        result = base_model(X_input)
        model_results.append(result)

    X = Average()(model_results)
    model = Model(inputs = model_inputs,outputs = X)
  
    return model

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    dot = np.dot(u,v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    cosine_similarity = dot/(norm_u*norm_v)
    
    return cosine_similarity

  
def triplet_loss(y_true, y_pred, alpha = 0.35):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    total_lenght = 64*3
    anchor, positive, negative = y_pred[:,0:int(total_lenght*1/3)],y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)],y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    basic_loss = pos_dist-neg_dist+alpha
    loss = K.sum(K.maximum(basic_loss,0.0))
    
    return loss
  
def base_model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    X = Conv1D(196,kernel_size = 15, strides = 4)(X_input)
    X = Activation('relu')(X)                                
    X = Dropout(rate = 0.2)(X)                                 
    
    X = LSTM(units = 128, return_sequences = True)(X_input)               
    X = Dropout(rate = 0.2)(X)                                 
    
    X = LSTM(units = 128, return_sequences = True)(X)                         
    X = Dropout(rate = 0.2)(X)                                
    
    X = LSTM(units = 128)(X)                                 
    X = Dropout(rate = 0.2)(X)                               
    
    X = Dense(64)(X)
    
    base_model = Model(inputs = X_input, outputs = X)

    return base_model  
  
def speech_model(input_shape, average_model):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)
    base_model -- model to be used to call the inputs

    Returns:
    model -- Keras model instance
    """
    
    #get triplets vectors
    input_anchor = []
    input_positive = []
    input_negative = []
    for i in range(15):
        X_input = Input(shape = input_shape, name = "Input"+ str(i))
        if (i < 5):
            input_anchor.append(X_input)
        elif (i < 10):
            input_positive.append(X_input)
        elif(i < 15):
            input_negative.append(X_input)
        
    vec_anchor = average_model(input_anchor)
    vec_positive = average_model(input_positive)
    vec_negative = average_model(input_negative)
    
    #Concatenate vectors vec_positive, vec_negative
    concat_layer = concatenate([vec_anchor,vec_positive,vec_negative], axis = -1, name='concat_layer')
    
    model = Model(inputs = input_anchor + input_positive + input_negative, outputs = concat_layer, name = 'speech_to_vec')
    #model = Model(inputs = input_anchor + input_positive + input_negative, outputs = [vec_anchor,vec_positive,vec_negative], name = 'speech_to_vec')
    #model = Model(inputs = [input_anchor,input_positiv], outputs=vec_anchor)
    
    
    return model  

In [0]:
d = 5
Tx = 318
n_freq = 101

base_model = base_model(input_shape = (n_freq, Tx))
average_model = five_average_model(input_shape = (n_freq, Tx), base_model = base_model)
speech_model = speech_model(input_shape = (n_freq, Tx), average_model = average_model) 
speech_model.compile(loss=triplet_loss, optimizer='adam', metrics=["accuracy"])

In [0]:
print (X.shape)
X_train, X_test = [], []
Y_train, Y_test = [], []
for i in range (X.shape[0]):
    if np.random.random() < 0.8:
        X_train.append(X[i])
        Y_train.append(Y[i])
    else:
        X_test.append(X[i])
        Y_test.append(Y[i])
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
print (X_train.shape)
print (X_test.shape)
        
    

In [0]:
X_train = reshape_X(X_train)
X_test = reshape_X(X_test)


In [0]:
print(X_train.shape)
print(X_test.shape)
!mkdir preprocessed
np.save('./preprocessed/X_train',X_train)
np.save('./preprocessed/X_test',X_test)
np.save('./preprocessed/Y_train',Y_train)
np.save('./preprocessed/Y_test',Y_test)
print (X.shape)
print (Y.shape)

np.save('./preprocessed/X_total',X)
np.save('./preprocessed/Y_total',Y)

In [0]:
X_train = np.load('./preprocessed/X_train.npy')
X_test = np.load('./preprocessed/X_test.npy')
Y_train = np.load('./preprocessed/Y_train.npy')
Y_test = np.load('./preprocessed/Y_test.npy')
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print(Y_test.shape)

In [0]:
def average_emb(X_predicts,Y_predicts):
    average_embeddings = {}
    count_embeddings = {}
    for i in range(len(X_predicts)):
        if Y_predicts[i] not in average_embeddings:
            average_embeddings[Y_predicts[i]] = X_predicts[i]
            count_embeddings[Y_predicts[i]] = 1
        else:
            average_embeddings[Y_train[i]] += X_predicts[i]
            count_embeddings[Y_predicts[i]] += 1

    for key, value in average_embeddings.items():
        average_embeddings[key] = average_embeddings[key]/count_embeddings[key]
    return average_embeddings

  
def get_acurracy(X_predicts):
    average_embeddings = average_emb(X_predicts,Y_train)
    count = 0
    bad_count = 0
    for i in range(len(X_predicts)):
    best_result = -1
    best_val = 0
    for key, value in average_embeddings.items():
        result = cosine_similarity(X_predicts[i],value)
        #print (key, Y_test[i],result)
        if result > best_result:
            best_result = result
            best_val = key

    if (best_val == Y_train[i]):
        count+=1
    else:
        bad_count+=1

    return (count/(count+bad_count))


def make_matrix(X_train,Y_train,average_model):
    print('making matrix')
    X_predicts = average_model.predict(X_train,batch_size = 16,verbose = 1)
    accuracy = get_acurracy(X_predicts)
    matrix = np.zeros((len(X_train[0]),len(X_train[0])))
    for i in range(len(X_predicts)):
        matrix[i][i] = 1.0
        for j in range(i+1,len(X_predicts)):
            matrix[i][j] = cosine_similarity(X_predicts[i],X_predicts[j])
            matrix[j][i] = matrix[i][j]
    return accuracy,matrix

def make_worse_triplets(matrix,X_train,Y_train):
    anchors = np.zeros((5,len(X_train[0]),101,318))
    positives = np.zeros((5,len(X_train[0]),101,318))
    negatives = np.zeros((5,len(X_train[0]),101,318))
    for x in range(len(matrix)):
        lowest = 1.5
        highest = -1.5
        anchor = X_train[:,x:x+1]
        for y in range(len(matrix)):
            if matrix[x][y] < lowest and (Y_train[x] == Y_train[y]):
                lowest = matrix[x][y]
                positive = y
            elif matrix[x][y] > highest and Y_train[x] != Y_train[y]:
                highest = matrix[x][y]
                negative = y
        anchors[:,x,:,:] = X_train[:,x,:,:]
        positives[:,x,:,:] = X_train[:,positive,:,:]
        negatives[:,x,:,:] = X_train[:,negative,:,:]
        #print (x,'/',len(matrix))
    return anchors,positives,negatives
   

In [0]:
epochs = 1000
batch_size = 32
Y_dummy = np.empty((batch_size, 3))
messages= []
accuracies = []

In [0]:
for i in range(epochs):
    print ("epoch",i)
    accuracy,matrix = make_matrix(X_train.tolist(),Y_train,bas e_model)
    print ("made matrix")
    print("acurracy",accuracy)
    accuracies += [accuracy]
    anchors, positives, negatives = make_worse_triplets(matrix,X_train,Y_train)
    start_idx = 0
    mean_loss = 0
    while (start_idx + batch_size < len(anchors[0])):
        batch_x = anchors[:,start_idx:start_idx + batch_size].tolist()+positives[:,start_idx:start_idx + batch_size].tolist()+negatives[:,start_idx:start_idx + batch_size].tolist()
        message = speech_model.train_on_batch(x = batch_x, y = Y_dummy)
        mean_loss+= message[0]
        messages+= [message]
        if ((start_idx/batch_size)%3 == 0):
            print ("start_idx:",start_idx, ",loss:",message)
        start_idx+=batch_size
    if (len(anchors[0])%batch_size != 0):
        batch_x = anchors[:,-batch_size:].tolist()+positives[:,-batch_size:].tolist()+negatives[:,-batch_size:].tolist()
        speech_model.train_on_batch(x = batch_x, y = Y_dummy)
    print("average_loss:",(mean_loss/(len(anchors[0])/batch_size+1)))

In [0]:
messages_donwload = np.array(messages)
acc_down = np.array(accuracies)
print (messages_donwload.shape)
print (acc_down.shape)

In [0]:
np.save('messages_download_first_133',messages_donwload)
np.save('acc_download_first_133',acc_down)

In [0]:
base_model.save("base_model_first_133.h5")

In [0]:
from google.colab import files
files.download('base_model_first_133.h5')
files.download('messages_download_first_133.npy')
files.download('acc_download_first_133.npy')

In [0]:
from google.colab import files
files.upload()


In [0]:
from tensorflow.keras.models import load_model
base_model_final = load_model('./finalv1.h5')



In [0]:
average_model = five_average_model(input_shape = (n_freq, Tx), base_model = base_model_final)


In [0]:
X_test_predict = average_model.predict(X_test.tolist(),batch_size = 8,verbose = 1)

In [0]:
X_train_predict = average_model.predict(X_train.tolist(),batch_size = 4,verbose = 1)

In [0]:
average_embeddings = average_emb(X_train_predict,Y_train)

In [0]:
count = 0
bad_count = 0
for i in range(len(X_test_predict)):
  
    best_result = -1
    best_val = 0
    for key, value in average_embeddings.items():
        result = cosine_similarity(X_test_predict[i],value)
        if result > best_result:
            best_result = result
            best_val = key
    if (best_val == Y_test[i]):
        count+=1
    else:
        bad_count+=1
print (count, bad_count)
print (count/(count+bad_count))


In [0]:
for i in range(len(X_train_predict)):
    best_result = -1
    best_val = 0
    for key, value in average_embeddings.items():
        result = cosine_similarity(X_train_predict[i],value)
        if result > best_result:
            best_result = result
            best_val = key
            
    if (best_val == Y_train[i]):
        count+=1
    else:
        bad_count+=1
print (count, bad_count)
print (count/(count+bad_count))
